In [1]:
import cv2
import numpy as np
import os
import mediapipe as mp
import sys

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

from closet_module import capture_module as cm
from closet_module import pose_module as pm
from closet_module import synthesis_module as sm
from closet_module import yolo_module as ym
from closet_module import cut_img_module as cim
from closet_module import grab_cut_module as gm
from closet_module import resize as re
from closet_module import color as cr
from closet_module import recommand as rc


from PyQt5.uic import loadUi
from PyQt5 import *
from PyQt5 import QtGui
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *

In [2]:
def centroid_histogram(clt):
    # 서로 다른 군집 수를 잡고 각 군집에 할당된 픽셀 수를 기반으로 히스토그램을 만듭니다.
    numLabels = np.arange(0, len(np.unique(clt.labels_)) + 1)
    (hist, _) = np.histogram(clt.labels_, bins=numLabels)

    # 히스토그램을 정규화하여 합계가 1이 되도록 하다
    hist = hist.astype("float")
    hist /= hist.sum()

    # return the histogram
    return hist

def plot_colors(hist, centroids):
    # 각 색상의 상대 빈도를 나타내는 막대 차트를 초기화합니다.
    bar = np.zeros((50, 300, 3), dtype="uint8")
    startX = 0
    color_name=set()
    # 각 클러스터의 백분율 및 각 클러스터의 색상에 대한 반복
    for (percent, color) in zip(hist, centroids):
        # 각 군집의 상대 백분율 그림
        endX = startX + (percent * 300)
        cv2.rectangle(bar, (int(startX), 0), (int(endX), 50), color.astype("uint8").tolist(), -1)
        startX = endX
        color_bar=color.astype("uint8").tolist()
        if int(percent*100)>15:
            test=cr.color(color_bar)
            color_name.add(test.find_color())
    # return the bar chart
    return color_name

In [3]:
#메인메뉴 UI  
class MainWindow(QWidget):
    def __init__(self):
        super(MainWindow,self).__init__()
        loadUi("main_menu.ui",self)
        self.setGeometry(300, 300, 233, 196)
        self.bt_1.clicked.connect(self.goto_capture_bt)
        self.bt_2.clicked.connect(self.syn_avatar)
        self.bt_3.clicked.connect(self.save_cloth)
        self.bt_4.clicked.connect(self.goto_category)
        self.bt_5.clicked.connect(self.cody)
        self.show()
        
    #아바타 생성        
    def goto_capture_bt(self):
        stack.setCurrentIndex(1)#아바타 캡쳐 UI
        cm.running = True
        cm.cap()
        
    #아바타 합성
    def syn_avatar(self):
        stack.setCurrentIndex(6) #합성 UI
        
    #옷 저장    
    def save_cloth(self):
        global mode
        mode = 5
        stack.setCurrentIndex(3) #아바타 뷰어 UI
        imageviewer.show(mode)
        
    #옷장 열기   
    def goto_category(self):
        global mode
        mode=6
        stack.setCurrentIndex(2) #카테고리 UI
        
    #코디 추천    
    def cody(self):
        global mode
        mode = 7
        stack.setCurrentIndex(2) #카테고리
##################################################################################    
#아바타 촬영 UI   
class capture_bt(QWidget):
    def __init__(self):
        super(capture_bt,self).__init__()
        loadUi("capture_bt.ui",self)
        self.check.clicked.connect(self.goto_MainWindow)
        self.check_2.clicked.connect(self.capture)
        self.setGeometry(300, 300, 132, 92)
        
    #아바타 촬영
    def capture(self):
        cm.picture = 1
        stack.setCurrentIndex(0)
        
    #메인메뉴로 돌아가기
    def goto_MainWindow(self):
        cm.running = False
        stack.setCurrentIndex(0)#메인메뉴 UI 
##################################################################################        
#카테고리선택 UI        
class category(QWidget):
    def __init__(self):
        super(category,self).__init__()
        loadUi("category2.ui",self)
        self.check.clicked.connect(self.goto_show_img)
        self.bt_1.clicked.connect(self.choice_hoody)
        self.bt_2.clicked.connect(self.choice_coat)
        self.bt_3.clicked.connect(self.choice_longsleeve)
        self.bt_4.clicked.connect(self.choice_shirts)
        self.bt_5.clicked.connect(self.choice_shortsleeve)
        self.bt_6.clicked.connect(self.choice_blazer)
        self.bt_7.clicked.connect(self.choice_blouson)
        self.bt_8.clicked.connect(self.choice_knit)
        self.bt_9.clicked.connect(self.choice_leatherjacket)
        self.bt_10.clicked.connect(self.choice_cardigan)
        self.bt_11.clicked.connect(self.choice_fieldjumper)
        self.bt_12.clicked.connect(self.choice_fleece)
        self.bt_13.clicked.connect(self.choice_vest)
        self.bt_14.clicked.connect(self.choice_trucker)
        self.bt_15.clicked.connect(self.choice_padding)
        self.bt_16.clicked.connect(self.choice_mustang)
        self.bt_17.clicked.connect(self.choice_sweatshirts)
        self.bt_18.clicked.connect(self.choice_skirt)
        self.bt_19.clicked.connect(self.choice_shorts)
        self.bt_20.clicked.connect(self.choice_sweatpants)
        self.bt_21.clicked.connect(self.choice_trouser)
        self.bt_22.clicked.connect(self.choice_widejean)
        self.bt_23.clicked.connect(self.choice_widetrouser)
        self.bt_24.clicked.connect(self.choice_jean)
        self.bt_25.clicked.connect(self.choice_dress)
        self.bt_26.clicked.connect(self.choice_sandal)
        self.bt_27.clicked.connect(self.choice_boots)
        self.bt_28.clicked.connect(self.choice_slipon)
        self.bt_29.clicked.connect(self.choice_sneakers)
        self.bt_30.clicked.connect(self.choice_dressshoes)
        self.bt_31.clicked.connect(self.choice_runningshoes)
        self.setGeometry(300, 300, 644, 363)
        self.show()
        
    #이미지 뷰어 UI로 넘어가기
    def goto_show_img(self):
        global mode
        if (mode == 6) or (mode == 7):
            pass
        else:
            mode = 3
        imageviewer.show(mode)
        stack.setCurrentIndex(3)   #이미지 뷰어 UI

    #버튼 입력    
    def choice_hoody(self):
        global choice_clothes
        choice_clothes='hoody'
        
    def choice_coat(self):
        global choice_clothes
        choice_clothes='coat'
    
    def choice_longsleeve(self):
        global choice_clothes
        choice_clothes='long sleeve'
        
    def choice_shirts(self):
        global choice_clothes
        choice_clothes='shirts'
        
    def choice_shortsleeve(self):
        global choice_clothes
        choice_clothes='short sleeve'
        
    def choice_blazer(self):
        global choice_clothes
        choice_clothes='blazer'
        
    def choice_blouson(self):
        global choice_clothes
        choice_clothes='blouson'
        
    def choice_knit(self):
        global choice_clothes
        choice_clothes='knit'
          
    def choice_leatherjacket(self):
        global choice_clothes
        choice_clothes='leather jacket'
        
    def choice_cardigan(self):
        global choice_clothes
        choice_clothes='cardigan'
        
    def choice_fieldjumper(self):
        global choice_clothes
        choice_clothes='field jumper'
        
    def choice_fleece(self):
        global choice_clothes
        choice_clothes='fleece'
       
    def choice_vest(self):
        global choice_clothes
        choice_clothes='vest'
        
    def choice_trucker(self):
        global choice_clothes
        choice_clothes='trucker'
          
    def choice_padding(self):
        global choice_clothes
        choice_clothes='padding'
        
    def choice_mustang(self):
        global choice_clothes
        choice_clothes='mustang'
        
    def choice_sweatshirts(self):
        global choice_clothes
        choice_clothes='sweat shirts'
        
    def choice_skirt(self):
        global choice_clothes
        choice_clothes='skirt'
           
    def choice_shorts(self):
        global choice_clothes
        choice_clothes='shorts'
        
    def choice_sweatpants(self):
        global choice_clothes
        choice_clothes='sweat pants'
         
    def choice_trouser(self):
        global choice_clothes
        choice_clothes='trouser'
        
    def choice_widejean(self):
        global choice_clothes
        choice_clothes='wide jean'
      
    def choice_widetrouser(self):
        global choice_clothes
        choice_clothes='wide trouser'
        
    def choice_jean(self):
        global choice_clothes
        choice_clothes='choice jean'
         
    def choice_dress(self):
        global choice_clothes
        choice_clothes='dress'
        
    def choice_sandal(self):
        global choice_clothes
        choice_clothes='sandal'
        
    def choice_boots(self):
        global choice_clothes
        choice_clothes='boots'
        
    def choice_slipon(self):
        global choice_clothes
        choice_clothes='slip on'
         
    def choice_sneakers(self):
        global choice_clothes
        choice_clothes='sneakers'
        
    def choice_dressshoes(self):
        global choice_clothes
        choice_clothes='dress shoes'
        
    def choice_runningshoes(self):
        global choice_clothes
        choice_clothes='running shoes'
        
##################################################################################        
#이미지 뷰어 UI 
class ImageViewer(QWidget):
    def __init__(self):
        super(ImageViewer,self).__init__()
        loadUi("imageviewer.ui",self)
        self.setGeometry(300, 300,  604, 482)
        self.next_bt.clicked.connect(self.next_img)
        self.pre_bt.clicked.connect(self.pre_img)
        self.choice_bt.clicked.connect(self.choice_img)
        self.pre_menu.clicked.connect(self.goto_pre_menu)
        self.close_bt.clicked.connect(self.goto_MainWindow)
        self.i=0
        self.path=''
        self.img_list=[]
        self.qPixmapFileVar = QPixmap()
        
    def show(self,mode):
        global choice_clothes
        if (mode == 2) or (mode ==5)or (mode ==8):
            self.path='C:\\Users\\lky\\DeepLearning\\smart_closet\\img_lib\Avatar'
        elif (mode == 3) or (mode == 4) or (mode==6) or (mode==7):
            self.path='C:\\Users\\lky\\DeepLearning\\smart_closet\\img_lib\\cut_img\\'+choice_clothes
        
        self.img_list=os.listdir(self.path)
        self.qPixmapFileVar.load(self.path+'\\'+self.img_list[self.i])
        self.qPixmapFileVar = self.qPixmapFileVar.scaledToWidth(500)
        self.label.setPixmap(self.qPixmapFileVar)
    
    #다음 이미지
    def next_img(self):
        if self.i < len(self.img_list)-1:
            self.i+=1
            self.qPixmapFileVar = QPixmap()
            self.qPixmapFileVar.load(self.path+'\\'+self.img_list[self.i])
            self.qPixmapFileVar = self.qPixmapFileVar.scaledToWidth(500)
            self.label.setPixmap(self.qPixmapFileVar)
            
    #이전 이미지        
    def pre_img(self):
        if self.i > 0:
            self.i-=1
            self.qPixmapFileVar = QPixmap()
            self.qPixmapFileVar.load(self.path+'\\'+self.img_list[self.i])
            self.qPixmapFileVar = self.qPixmapFileVar.scaledToWidth(500)
            self.label.setPixmap(self.qPixmapFileVar)
            
    
    #이미지 선택하기        
    def choice_img(self):
        global avatar_img
        global cloth_img
        global mode
        #합성단계시
        if mode == 2:
            stack.setCurrentIndex(6) #합성 UI
            avatar_path=self.path+'\\'+self.img_list[self.i]
            avatar_img = cv2.imread(avatar_path)
            syn_cloth.show(avatar_img)
        elif mode == 3:
            stack.setCurrentIndex(6)#합성 UI
            cloth_path = self.path+'\\'+self.img_list[self.i]
            cloth_img = cv2.imread(cloth_path)
        elif mode ==5:
            avatar_path=self.path+'\\'+self.img_list[self.i]
            avatar_img = cv2.imread(avatar_path)
            stack.setCurrentIndex(4) #top,bottom UI
            
        elif mode ==7:
            cloth_keward=self.img_list[self.i]
            
            cloth_keward=cloth_keward.replace('.png','')
            cloth_keward = cloth_keward.replace('0','')
            cloth_keward = cloth_keward.replace('1','')
            cloth_keward = cloth_keward.replace('2','')
            cloth_keward = cloth_keward.replace('3','')
            cloth_keward = cloth_keward.replace('4','')
            cloth_keward = cloth_keward.replace('5','')
            cloth_keward = cloth_keward.replace('6','')
            cloth_keward = cloth_keward.replace('7','')
            cloth_keward = cloth_keward.replace('8','')
            cloth_keward = cloth_keward.replace('9','')
            outer=rc.recommend_outer(cloth_keward)
            top=rc.recommend_top(cloth_keward)
            bottom=rc.recommend_bottom(cloth_keward)
            try:
                outer_path=rc.find_outer_path(outer)
            except:
                pass
            try:
                top_path=rc.find_top_path(top)
            except:
                pass
            try:
                bottom_path=rc.find_bottom_path(bottom)
            except:
                pass
            
            try:
                recommand.show1(outer_path)
            except:
                pass
            try:
                recommand.show2(top_path)
            except:
                pass
            try:
                recommand.show3(bottom_path)
            except:
                pass
         
            stack.setCurrentIndex(7)#추천UI
            
        elif mode ==8:
            avatar_path=self.path+'\\'+self.img_list[self.i]
            avatar_img = cv2.imread(avatar_path)
            stack.setCurrentIndex(7)#추천UI
            
        else:
            pass
            
    #메인 메뉴로 돌아가기
    def goto_MainWindow(self):
        set_return()
        stack.setCurrentIndex(0) #메인메뉴 UI  
        
    #이전 메뉴으로 돌아가기    
    def goto_pre_menu(self):
        global mode
        if mode ==2:
            stack.setCurrentIndex(6)#합성UI
        elif (mode == 3) or (mode ==6) or (mode == 7):
            stack.setCurrentIndex(2) #카테고리 UI
        
##################################################################################        
#상,하의 선택 UI       
class Top_Bottom(QWidget):
    def __init__(self):
        super(Top_Bottom,self).__init__()
        loadUi("choice_top_bottom.ui",self)
        self.bt1.clicked.connect(self.top)
        self.bt2.clicked.connect(self.bottom)
        self.setGeometry(300, 300, 123, 116)

    def top(self):
        global top_sign
        global avatar_img
        global yolo_img
        global yolo_name
        global pose_cap_img
        global mode
        if mode == 4:
            top_sign=True
            stack.setCurrentIndex(6) #합성 UI
            
        elif mode == 5:
            pm.check_image_angle(avatar_img)
            pose_cap_img=pm.video_angle(top_sign)
            yolo=ym.yolo(pose_cap_img,True)
            yolo_img=yolo.detect_cloth_img()
            yolo_name=yolo.detect_cloth_label()
            yoloshow.show()
            stack.setCurrentIndex(8) #yolo UI
           
    def bottom(self):
        global top_sign
        global avatar_img
        global yolo_img
        global yolo_name
        global pose_cap_img
        global mode
        if mode == 4:
            top_sign=False
            stack.setCurrentIndex(6) #합성 UI
            
        elif mode == 5:
            pm.check_image_angle(avatar_img)
            pose_cap_img=pm.video_angle(top_sign)
            yolo=ym.yolo(pose_cap_img,True)
            yolo_img=yolo.detect_cloth_img()
            yolo_name=yolo.detect_cloth_label()
            yoloshow.show()
            stack.setCurrentIndex(8) #yolo UI
##################################################################################                 
#색상 선택 UI    
class Color(QWidget):
    def __init__(self):
        super(Color,self).__init__()
        loadUi("color.ui",self)
        self.setGeometry(300, 300, 276, 373)
        
        self.check.clicked.connect(self.save_cloth)
        
        self.beige.clicked.connect(self.beige_color)
        self.black.clicked.connect(self.black_color)
        self.blue.clicked.connect(self.blue_color)
        self.brown.clicked.connect(self.brown_color)
        self.grey.clicked.connect(self.grey_color)
        self.khaki.clicked.connect(self.khaki_color)
        self.navy.clicked.connect(self.navy_color)
        self.orange.clicked.connect(self.orange_color)
        self.green.clicked.connect(self.green_color)
        self.pink.clicked.connect(self.pink_color)
        self.purple.clicked.connect(self.purple_color)
        self.red.clicked.connect(self.red_color)
        self.yellow.clicked.connect(self.yellow_color)
        self.white.clicked.connect(self.white_color)
    
    def text_color(self):
        global color_name
        self.color_list=list(color_name)
        text=''
        for i in range(len(self.color_list)):
            text=text+f'{i+1}번: '+self.color_list[i]+' '
        self.label.setText(text)
        

    def beige_color(self):
        global choice_color
        choice_color = 'beige'
    
    def black_color(self):
        global choice_color
        choice_color = 'black'
    
    def blue_color(self):
        global choice_color
        choice_color = 'blue'
    
    def brown_color(self):
        global choice_color
        choice_color = 'brown'
    
    def grey_color(self):
        global choice_color
        choice_color = 'grey'
        
    def khaki_color(self):
        global choice_color
        choice_color = 'khaki'
    
    def navy_color(self):
        global choice_color
        choice_color = 'navy'
        
    def orange_color(self):
        global choice_color
        choice_color = 'orange'
        
    def green_color(self):
        global choice_color
        choice_color = 'green'
    
    def pink_color(self):
        global choice_color
        choice_color = 'pink'
    
    def purple_color(self):
        global choice_color
        choice_color = 'purple'
        
    def red_color(self):
        global choice_color
        choice_color = 'red'
    
    def yellow_color(self):
        global choice_color
        choice_color = 'yellow'
        
    def white_color(self):
        global choice_color
        choice_color = 'white'
        
    def save_cloth(self):
        global cloth_category
        global cut_img
        save_path='C:\\Users\\lky\\DeepLearning\\smart_closet\\img_lib\\cut_img\\'
        i=1
        while(True):
            file = save_path+cloth_category+'\\'+choice_color+' '+cloth_category+f'{i}'+'.png'
            if os.path.isfile(file):
                i+=1
                continue
            cv2.imwrite(save_path+cloth_category+'\\'+choice_color+' '+cloth_category+f'{i}'+'.png',cut_img)
            break
        stack.setCurrentIndex(0)#메인메뉴 UI
        set_return()
##################################################################################        
#옷 합성UI        
class Syn_Cloth(QWidget):
    def __init__(self):
        super(Syn_Cloth,self).__init__()
        loadUi("syn_cloth.ui",self)
        self.close.clicked.connect(self.goto_MainWindow)
        self.choice_bt.clicked.connect(self.choice)
        self.choice_av.clicked.connect(self.choice_avatar)
        self.syn_bt.clicked.connect(self.syn)
        self.reset_syn.clicked.connect(self.clear_syn)
        self.setGeometry(300, 300, 839, 620)
    
    #메인으로 돌아가기
    def goto_MainWindow(self):
        set_return()
        stack.setCurrentIndex(0) #메인메뉴 UI  
        
    def show(self,img):
        #화면에 표시
        syn_avatar_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h,w,c = syn_avatar_img.shape
        self.qImg = QtGui.QImage(syn_avatar_img.data, w, h, w*c, QtGui.QImage.Format_RGB888)
        self.pixmap = QtGui.QPixmap.fromImage(self.qImg)
        self.pixmap = self.pixmap.scaledToWidth(670)
        self.label.setPixmap(self.pixmap)
        
    
    #이미지 합성    
    def syn(self):
        global avatar_img
        global cloth_img
        global mode
        mode = 4
        stack.setCurrentIndex(4) #top.bottom UI
        #이미지,위치 변수
        syn_avatar_img=avatar_img
        syn_cloth_img=cloth_img
        position=pm.check_landmark(syn_avatar_img)
        #이미지 합성
        if top_sign:
            syn_avatar_img = sm.synthesis(position[6][0]-50,position[0][1]-80,syn_avatar_img,syn_cloth_img)
        else:
            syn_avatar_img = sm.synthesis(position[2][0]-50,position[1][1]-80,syn_avatar_img,syn_cloth_img)
        
        self.show(syn_avatar_img)
        

    #이미지 초기화    
    def clear_syn(self):
        pass
    
    #의상 선택
    def choice(self):
        global mode
        mode = 3
        stack.setCurrentIndex(2) #카테고리 UI
    
    #아바타 선택    
    def choice_avatar(self):
        global mode
        mode = 2
        stack.setCurrentIndex(3)#아바타 뷰어 UI
        imageviewer.show(mode)
        
##################################################################################
class Recommand(QWidget):
    def __init__(self):
        super(Recommand,self).__init__()
        loadUi("recommand.ui",self)
        self.setGeometry(100, 100, 1268, 891)
        
        self.a_1.clicked.connect(self.before1)
        self.b_1.clicked.connect(self.before2)
        self.c_1.clicked.connect(self.before3)
        self.d_1.clicked.connect(self.before4)
        
        self.a_2.clicked.connect(self.after1)
        self.b_2.clicked.connect(self.after2)
        self.c_2.clicked.connect(self.after3)
        self.d_2.clicked.connect(self.after4)
        
        self.a_3.clicked.connect(self.choice1)
        self.b_3.clicked.connect(self.choice2)
        self.c_3.clicked.connect(self.choice3)
        self.d_3.clicked.connect(self.choice4)
        
        self.view.clicked.connect(self.show_cody)
        self.avatar.clicked.connect(self.choice_avatar)
        self.ex.clicked.connect(self.goto_mainmenu)
        
        self.i1=0
        self.path1=''
        self.img_list1=[]
        self.qPixmapFileVar1 = QPixmap()
        
        self.i2=0
        self.path2=''
        self.img_list2=[]
        self.qPixmapFileVar2 = QPixmap()
        
        self.i3=0
        self.path3=''
        self.img_list3=[]
        self.qPixmapFileVar3 = QPixmap()
        
        self.i4=0
        self.path4=''
        self.img_list4=[]
        self.qPixmapFileVar4 = QPixmap()
        
    def show1(self,path):
        #화면에 표시
        self.img_list1=path
        self.qPixmapFileVar1.load(self.img_list1[self.i1])
        self.qPixmapFileVar1 = self.qPixmapFileVar1.scaledToWidth(500)
        self.label.setPixmap(self.qPixmapFileVar1)
         
    def show2(self,path):
        #화면에 표시
        self.img_list2=path
        self.qPixmapFileVar2.load(self.img_list2[self.i2])
        self.qPixmapFileVar2 = self.qPixmapFileVar2.scaledToWidth(400)
        self.label_2.setPixmap(self.qPixmapFileVar2)
        
    def show3(self,path):
        #화면에 표시
        self.img_list3=path
        self.qPixmapFileVar3.load(self.img_list3[self.i3])
        self.qPixmapFileVar3 = self.qPixmapFileVar3.scaledToWidth(400)
        self.label_3.setPixmap(self.qPixmapFileVar3)
        
    def show4(self,path):
        #화면에 표시
        self.img_list4=path
        self.qPixmapFileVar4.load(self.img_list4[self.i4])
        self.qPixmapFileVar4 = self.qPixmapFileVar4.scaledToWidth(400)
        self.label_4.setPixmap(self.qPixmapFileVar4)
    
      
    def before1(self):
        if self.i1 > 0:
            self.i1-=1
            self.qPixmapFileVar1 = QPixmap()
            self.qPixmapFileVar1.load(self.img_list1[self.i1])
            self.qPixmapFileVar1 = self.qPixmapFileVar1.scaledToWidth(400)
            self.label.setPixmap(self.qPixmapFileVar1)
    def before2(self):
        if self.i2 > 0:
            self.i2-=1
            self.qPixmapFileVar2 = QPixmap()
            self.qPixmapFileVar2.load(self.img_list2[self.i2])
            self.qPixmapFileVar2 = self.qPixmapFileVar2.scaledToWidth(400)
            self.label_2.setPixmap(self.qPixmapFileVar2)
    def before3(self):
        if self.i3 > 0:
            self.i3-=1
            self.qPixmapFileVar3 = QPixmap()
            self.qPixmapFileVar3.load(self.img_list3[self.i3])
            self.qPixmapFileVar3 = self.qPixmapFileVar3.scaledToWidth(400)
            self.label_3.setPixmap(self.qPixmapFileVar3)
    def before4(self):
        if self.i4 > 0:
            self.i4-=1
            self.qPixmapFileVar4 = QPixmap()
            self.qPixmapFileVar4.load(self.img_list4[self.i4])
            self.qPixmapFileVar4 = self.qPixmapFileVar4.scaledToWidth(400)
            self.label_4.setPixmap(self.qPixmapFileVar4)
    
    def after1(self):
        if self.i1 < len(self.img_list1)-1:
            self.i1+=1
            self.qPixmapFileVar1 = QPixmap()
            self.qPixmapFileVar1.load(self.img_list[self.i1])
            self.qPixmapFileVar1 = self.qPixmapFileVar1.scaledToWidth(400)
            self.label.setPixmap(self.qPixmapFileVar1)
    def after2(self):
        if self.i2 < len(self.img_list2)-1:
            self.i2+=1
            self.qPixmapFileVar2 = QPixmap()
            self.qPixmapFileVar2.load(self.img_list2[self.i2])
            self.qPixmapFileVar2 = self.qPixmapFileVar2.scaledToWidth(400)
            self.label_2.setPixmap(self.qPixmapFileVar2)
    def after3(self):
        if self.i3 < len(self.img_list3)-1:
            self.i3+=1
            self.qPixmapFileVar3 = QPixmap()
            self.qPixmapFileVar3.load(self.img_list3[self.i3])
            self.qPixmapFileVar3 = self.qPixmapFileVar3.scaledToWidth(400)
            self.label_3.setPixmap(self.qPixmapFileVar3)
    def after4(self):
        if self.i4 < len(self.img_list4)-1:
            self.i4+=1
            self.qPixmapFileVar4 = QPixmap()
            self.qPixmapFileVar4.load(self.img_list4[self.i4])
            self.qPixmapFileVar4 = self.qPixmapFileVar4.scaledToWidth(400)
            self.label_4.setPixmap(self.qPixmapFileVar4)
    
    def choice1(self):
        cloth_path1 = self.img_list1[self.i1]
        self.cloth_img1 = cv2.imread(cloth_path1)
    def choice2(self):
        cloth_path2 = self.img_list2[self.i2]
        self.cloth_img2 = cv2.imread(cloth_path2)
    def choice3(self):
        cloth_path3 = self.img_list3[self.i3]
        self.cloth_img3 = cv2.imread(cloth_path3)
    def choice4(self):
        cloth_path4 = self.img_list4[self.i4]
        self.cloth_img4 = cv2.imread(cloth_path4)
    
    def show_cody(self):
        global avatar_img
        #이미지,위치 변수
        syn_avatar_img=avatar_img
        position=pm.check_landmark(syn_avatar_img)
        #외투 합성
        try:
            syn_cloth_img1=self.cloth_img1
            syn_avatar_img = sm.synthesis(position[6][0]-50,position[0][1]-80,syn_avatar_img,syn_cloth_img1)
        except:
            pass
        #상의 합성
        try:
            syn_cloth_img2=self.cloth_img2
            syn_avatar_img = sm.synthesis(position[6][0]-50,position[0][1]-80,syn_avatar_img,syn_cloth_img2)
        except:
            pass
        #하의 합성
        try:
            syn_cloth_img3=self.cloth_img3
            syn_avatar_img = sm.synthesis(position[2][0]-50,position[1][1]-80,syn_avatar_img,syn_cloth_img3)
        except:
            pass
        
        view_cody.show(syn_avatar_img)
        stack.setCurrentIndex(9)
        
    def choice_avatar(self):
        global mode
        mode =8
        stack.setCurrentIndex(3)#아바타 뷰어 UI
        imageviewer.show(mode)
        
    def goto_mainmenu(self):
        set_return()
        stack.setCurrentIndex(0)
##################################################################################
class YoloShow(QWidget):
    def __init__(self):
        super(YoloShow,self).__init__()
        loadUi("yolo.ui",self)
        self.setGeometry(300, 300, 646, 449)
        self.name_list=[]
        
    def show(self):
        global yolo_img
        global yolo_name
        
        img=yolo_img
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h,w,c = img.shape
        self.qImg = QtGui.QImage(img.data, w, h, w*c, QtGui.QImage.Format_RGB888)
        self.pixmap = QtGui.QPixmap.fromImage(self.qImg)
        self.pixmap = self.pixmap.scaledToWidth(670)
        self.label.setPixmap(self.pixmap)
        
        self.name_list=list(yolo_name)
        text=''
        for i in range(len(self.name_list)):
            text=text+f'{i+1}번: '+self.name_list[i]+' '
        
        self.label_2.setText(text)
        self.b_1.clicked.connect(self.one)
        self.b_2.clicked.connect(self.two)
        self.check.clicked.connect(self.ok)
        
    def one(self):
        global cloth_category
        try:
            cloth_category=self.name_list[0]
        except:
            pass
        
    def two(self):
        global cloth_category
        try:
            cloth_category=self.name_list[1]
        except:
            pass
        
    def three(self):
        global cloth_category
        try:
            cloth_category=self.name_list[2]
        except:
            pass
        
    def four(self):
        global cloth_category
        try:
            cloth_category=self.name_list[3]
        except:
            pass
        
    def five(self):
        global cloth_category
        try:
            cloth_category=self.name_list[4]
        except:
            pass
        
    def six(self):
        global cloth_category
        try:
            cloth_category=self.name_list[5]
        except:
            pass
             
    def ok(self):
        global pose_cap_img
        global cut_img
        global color_name
        img_box=pm.check_landmark(pose_cap_img)
        if top_sign:
            grab_img = gm.grabcut(pose_cap_img,img_box[6][0]-50,img_box[0][1]-70,img_box[7][0]+50,img_box[4][1]+10)
        else:
            grab_img = gm.grabcut(pose_cap_img,img_box[2][0]-50,img_box[1][1]-70,img_box[5][0]+50,img_box[5][1]+10)
        resize=re.resize(avatar_img,pose_cap_img,grab_img,top_sign)
        cut_img=resize.cut()
        
        a=int(cut_img.shape[0]/2)
        b=int(cut_img.shape[1]/2)

        src = cut_img
        #색상을 추출할 범위
        dst = src[a-70:a+70, b-70:b+70].copy()
        dst = dst.reshape((dst.shape[0] * dst.shape[1], 4))

        k = 3 # 추출할 색상 수
        clt = KMeans(n_clusters = k)
        clt.fit(dst)

        hist = centroid_histogram(clt)
        color_name = plot_colors(hist, clt.cluster_centers_)
        
        color.text_color()
        stack.setCurrentIndex(5) #컬러 UI
#############################################################################
#코디뷰UI
class View_Cody(QWidget):
    def __init__(self):
        super(View_Cody,self).__init__()
        loadUi("show_cody.ui",self)
        self.setGeometry(300, 300, 769, 708)
        self.b1.clicked.connect(self.back)
        self.b2.clicked.connect(self.goto_mainmenu)
    
    def show(self,syn_img):
        img=syn_img
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h,w,c = img.shape
        self.qImg = QtGui.QImage(img.data, w, h, w*c, QtGui.QImage.Format_RGB888)
        self.pixmap = QtGui.QPixmap.fromImage(self.qImg)
        self.pixmap = self.pixmap.scaledToWidth(600)
        self.label.setPixmap(self.pixmap)
    
    def back(self):
        stack.setCurrentIndex(7)
    
    def goto_mainmenu(self):
        set_return()
        stack.setCurrentIndex(0)
#############################################################################
#전역변수 초기화
def set_return():
    global mode
    global choice_clothes
    global avatar_img
    global cloth_img
    global yolo_img
    global yolo_name
    global pose_cap_img
    global top_sign
    global color_name
    global choice_color
    global cut_img
    global cloth_category
    
    choice_clothes=''
    mode=0
    avatar_img=[]
    cloth_img=[]
    yolo_img=[]
    yolo_name=''
    pose_cap_img=[]
    top_sign=True
    color_name=''
    choice_color=''
    cut_img=[]
    cloth_category=''
############################################################################# 
#메인 인스턴스 생성
app = QApplication(sys.argv)

#전역변수 초기값
top_sign=True
mode=0

#스택생성
stack = QStackedLayout()

#위젯생성
mainwindow = MainWindow()
stack.addWidget(mainwindow)#index0

capture_bt=capture_bt()
stack.addWidget(capture_bt)#index1

category=category()
stack.addWidget(category)#index2

imageviewer=ImageViewer()
stack.addWidget(imageviewer)#index3

top_bottom=Top_Bottom()
stack.addWidget(top_bottom)#index4

color=Color()
stack.addWidget(color)#index5

syn_cloth=Syn_Cloth()
stack.addWidget(syn_cloth)#index6

recommand=Recommand()
stack.addWidget(recommand)#index 7

yoloshow=YoloShow()
stack.addWidget(yoloshow)#index 8

view_cody=View_Cody()
stack.addWidget(view_cody)#index 9

#프로그램 종료
sys.exit(app.exec_())

SystemExit: 0

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
